In [10]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.signal import butter, lfilter, filtfilt

from asp.load import Trial

%matplotlib qt5

In [2]:
totalTrial = 1
t = []
for trialId in range(1,totalTrial+1):
    i = trialId-1
    t.append(Trial(2, trialId))
    t[i].readDecoder()
    t[i].readConductor()
    t[i].gaitSegmentation(check=False)
    t[i].readEEG()

In [4]:
def cutArray(arr, t_start, t_end):
# Summary: The first column of the input array is time, other columns are data. Cut part of this array based on start/end time
# prerequisite:
# Input: n-column array like rCurveHip, and the starting and ending time of segmentation
# Output: a segment of this array
# Date: 2017-7-28

    t_start_idx = np.argmax (arr[:,0]>t_start)
    t_end_idx = np.argmax (arr[:,0]>t_end)

    output = arr[t_start_idx:t_end_idx, :]

    return output

In [71]:
def eegBands(data, fs=100):
    f, Pxx = scipy.signal.welch(data, fs=fs, window='hanning')
    bands = []
    bands.append(np.where((f>0.1) & (f<4) ))
    bands.append(np.where((f>4)   & (f<8) ))
    bands.append(np.where((f>8)   & (f<12)))
    bands.append(np.where((f>12)  & (f<30)))
    bands.append(np.where((f>30)  & (f<49)))
    bandPowers = [sum(Pxx[x]) for x in bands]
    return bandPowers

In [99]:
def eegPowerTrainTest(eegMatrix, chan, fs=100):
    # Input: eegMatrix is the matrix without EOG channels. Basically the self.eeg 
    # Output: a 2*5 matrix representing the power in each band (normalized by signal length)
    # 2 rows: train, test. 5 columns: the five EEG bands
    eegTrain = cutArray(eegMatrix, t[i].events['tdmStart']+60, t[0].events['brainStart']-5)
    eegTest = cutArray(eegMatrix, t[i].events['brainStart']+2, t[0].events['brainEnd']-5)
    bandPowersTrain = np.array(eegBands(eegTrain[:, chan])).reshape((1,-1))/eegTrain.shape[0] # normalize by length 
    bandPowersTest  = np.array(eegBands(eegTest [:, chan])).reshape((1,-1))/eegTest.shape[0]
    bandPowers = np.concatenate((bandPowersTrain, bandPowersTest), axis=0)
    return bandPowers

In [101]:
eegPowerTrainTest(t[0].eeg, 15)

array([[ 0.02122463,  0.00094022,  0.00081939,  0.00276141,  0.00142677],
       [ 0.03134755,  0.00073329,  0.00066063,  0.00175776,  0.00074328]])